In [1]:
import Extractor
import pandas as pd
import sqlite3

conn1 = sqlite3.connect('Entertainment.db')
cur1 = conn1.cursor()
df1 = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn1)
conn1.close()

conn2 = sqlite3.connect('Entertainment101.db')
cur2 = conn2.cursor()
df2 = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn2)
conn2.close()

df = df1.append(df2) # 엔터테인먼트는 여기서 분할해서 저장해야함.

df['pk'] = range(1, len(df)+1)

conn3 = sqlite3.connect('Total_Ent.db')
cur3 = conn3.cursor()
cur3.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df.to_sql('head', conn3, if_exists='append', index=False)
conn3.close()

In [2]:
conn = sqlite3.connect('Total_Ent.db')
cur = conn.cursor()
df = pd.read_sql('SELECT * FROM head ORDER BY wdate DESC',conn)
df1 = df[:2000000]
df2 = df[2000000:4000000]
df3 = df[4000000:6000000]
df4 = df[6000000:8000000]
df5 = df[8000000:]

conn = sqlite3.connect('Ent_1.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df1.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_2.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df2.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_3.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df3.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_4.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df4.to_sql('head', conn, if_exists='append', index=False)
conn.close()

conn = sqlite3.connect('Ent_5.db')
cur = conn.cursor()
cur.executescript('''
            DROP TABLE IF EXISTS head;
            CREATE TABLE head(
                pk INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
                head TEXT NOT NULL,
                wdate TEXT NOT NULL,
                cdate TEXT NOT NULL,
                ref INTEGER NOT NULL,
                page INTEGER NOT NULL
            );
        ''')
df5.to_sql('head', conn, if_exists='append', index=False)
conn.close()

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('Ent_1_var.db')
df1 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_2_var.db')
df2 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_3_var.db')
df3 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_4_var.db')
df4 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('Ent_5_var.db')
df5 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

conn = sqlite3.connect('HP_var.db')
df6 = pd.read_sql('SELECT * FROM var', conn)
conn.close()

df = df1.append(df2).append(df3).append(df4).append(df5).append(df6)
conn = sqlite3.connect('var.db')
df.to_sql('var', conn, index=False)

In [1]:
import Extractor
import pandas as pd
import sqlite3

conn = sqlite3.connect('Ent_1.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head ORDER BY wdate DESC',conn)
conn.close()
df.drop_duplicates(keep='first', inplace=True)
ext = Extractor.Ext(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

# 변수 생성
statistic = ext.extract_statistic_value(sent)
r_rat = ext.extract_r_rat(sent,statistic)
statistic = ext.combine_var(statistic, r_rat)
conn = sqlite3.connect('Ent_1_var.db')
statistic.to_sql('var', conn)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1382555 from 1658304 sents. mem=0.680 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=7295360, mem=3.591 Gb
[Noun Extractor] batch prediction was completed for 322395 words
[Noun Extractor] checked compounds. discovered 343047 compounds
[Noun Extractor] postprocessing detaching_features : 3513 -> 3445
[Noun Extractor] postprocessing ignore_features : 3445 -> 3331
[Noun Extractor] postprocessing ignore_NJ : 3331 -> 3155
[Noun Extractor] 3155 nouns (343047 compounds) with min frequency=165
[Noun Extractor] flushing was done. mem=3.687 Gb                    
[Noun Extractor] 56.21 % eojeols are covered
training was done. used memory 4.379 Gb4.403 Gb
all cohesion probabilities was computed. # words = 144
all branching entropies was computed # words = 15447
al

all accessor variety was computed # words = 57895
training was done. used memory 4.406 Gb4.406 Gb
all cohesion probabilities was computed. # words = 621
all branching entropies was computed # words = 58311
all accessor variety was computed # words = 58311
training was done. used memory 4.406 Gb4.406 Gb
all cohesion probabilities was computed. # words = 621
all branching entropies was computed # words = 58459
all accessor variety was computed # words = 58459
training was done. used memory 4.406 Gb4.406 Gb
all cohesion probabilities was computed. # words = 625
all branching entropies was computed # words = 58762
all accessor variety was computed # words = 58762
training was done. used memory 4.406 Gb4.406 Gb
all cohesion probabilities was computed. # words = 627
all branching entropies was computed # words = 58966
all accessor variety was computed # words = 58966
training was done. used memory 4.406 Gb4.406 Gb
all cohesion probabilities was computed. # words = 630
all branching entropies

all accessor variety was computed # words = 67106
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 732
all branching entropies was computed # words = 67226
all accessor variety was computed # words = 67226
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 735
all branching entropies was computed # words = 67406
all accessor variety was computed # words = 67406
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 738
all branching entropies was computed # words = 67448
all accessor variety was computed # words = 67448
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 741
all branching entropies was computed # words = 67572
all accessor variety was computed # words = 67572
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 742
all branching entropies

all accessor variety was computed # words = 71519
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 830
all branching entropies was computed # words = 71645
all accessor variety was computed # words = 71645
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 835
all branching entropies was computed # words = 71723
all accessor variety was computed # words = 71723
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 842
all branching entropies was computed # words = 71827
all accessor variety was computed # words = 71827
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 843
all branching entropies was computed # words = 71929
all accessor variety was computed # words = 71929
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 848
all branching entropies

all branching entropies was computed # words = 76214
all accessor variety was computed # words = 76214
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 961
all branching entropies was computed # words = 76288
all accessor variety was computed # words = 76288
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 964
all branching entropies was computed # words = 76342
all accessor variety was computed # words = 76342
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 965
all branching entropies was computed # words = 76406
all accessor variety was computed # words = 76406
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilities was computed. # words = 968
all branching entropies was computed # words = 76460
all accessor variety was computed # words = 76460
training was done. used memory 4.407 Gb4.407 Gb
all cohesion probabilitie

...

[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 39.47 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 521 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=765, mem=1.190 Gb
[Noun Extractor] batch prediction was completed for 177 words
[Noun Extractor] checked compounds. discovered 1 compounds
[Noun Extractor] postprocessing detaching_features : 34 -> 34
[Noun Extractor] postprocessing ignore_features : 34 -> 32
[Noun Extractor] postprocessing ignore_NJ : 32 -> 32
[Noun Extractor] 32 nouns (1 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 32.55 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 556 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has bee

[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 32.30 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 781 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=1078, mem=1.190 Gb
[Noun Extractor] batch prediction was completed for 276 words
[Noun Extractor] checked compounds. discovered 2 compounds
[Noun Extractor] postprocessing detaching_features : 58 -> 58
[Noun Extractor] postprocessing ignore_features : 58 -> 57
[Noun Extractor] postprocessing ignore_NJ : 57 -> 57
[Noun Extractor] 57 nouns (2 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 34.04 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 634 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has be

[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 31.55 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 550 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=802, mem=1.190 Gb
[Noun Extractor] batch prediction was completed for 158 words
[Noun Extractor] checked compounds. discovered 3 compounds
[Noun Extractor] postprocessing detaching_features : 33 -> 33
[Noun Extractor] postprocessing ignore_features : 33 -> 31
[Noun Extractor] postprocessing ignore_NJ : 31 -> 31
[Noun Extractor] 31 nouns (3 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 38.28 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 653 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has bee

[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 36.97 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 784 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=1010, mem=1.190 Gb
[Noun Extractor] batch prediction was completed for 314 words
[Noun Extractor] checked compounds. discovered 5 compounds
[Noun Extractor] postprocessing detaching_features : 66 -> 66
[Noun Extractor] postprocessing ignore_features : 66 -> 63
[Noun Extractor] postprocessing ignore_NJ : 63 -> 62
[Noun Extractor] 62 nouns (5 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 35.25 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 212 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has be

[Noun Extractor] 79 nouns (12 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 34.89 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 700 from 1 sents. mem=1.190 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=919, mem=1.190 Gb
[Noun Extractor] batch prediction was completed for 225 words
[Noun Extractor] checked compounds. discovered 11 compounds
[Noun Extractor] postprocessing detaching_features : 55 -> 55
[Noun Extractor] postprocessing ignore_features : 55 -> 53
[Noun Extractor] postprocessing ignore_NJ : 53 -> 53
[Noun Extractor] 53 nouns (11 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.190 Gb                    
[Noun Extractor] 34.39 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 486 from 1 sents. mem=1.190 Gb                    
[Noun Extract